In [20]:
import os
from groq import Groq
from dotenv import load_dotenv
import pandas as pd

from datetime import datetime
import time
import requests
from urllib.parse import urljoin, urlparse
import logging
import os
import glob

In [22]:
df = pd.read_excel("output\\articles_scraped.xlsx")
#df.shape
df.columns.to_list()

['url',
 'title',
 'publish_date',
 'authors',
 'scraped_at',
 'keywords',
 'text',
 'text_length']

In [23]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
from transformers import GPT2TokenizerFast
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [24]:
from transformers import GPT2TokenizerFast
tokenizer = GPT2TokenizerFast.from_pretrained('Xenova/gpt-4o')

In [ ]:
from groq import Groq

client = Groq(
    api_key="GROQ_API_KEY",
)

In [37]:
from openai import OpenAI
client = OpenAI(api_key="OPENAI_API_KEY")

#### OpenAI (gpt4oMini)

In [ ]:
def get_articles_with_metadata(df, num_articles=5, offset=0):
    articles = df.iloc[offset:offset + num_articles][['url',
 'title',
 'publish_date',
 'authors',
 'scraped_at',
 'keywords',
 'text',
 'text_length']].to_dict(orient='records')
    return articles

# Example to obtain the first 5 articles
articles = get_articles_with_metadata(df, num_articles=5, offset=0)

# Using metadata to build article text
articles_text = "\n\n".join(
    [f"### Article {i+1}:\nTitle: {article['title']}\nURL: {article['url']}\nPublish_date: {article['publish_date']}\n"
     for i, article in enumerate(articles)]
)

# Dynamically generate the ‘Article Summaries’ section with numbers
article_summaries = "\n\n".join(
    [f"""### {i+1}. 
     
     **Title**: {article['title']}
   **Link to source**: {article['url']} 
   **Publish_date**: {article['publish_date']}
   **Summary**: Summarize the key points of the article in 2-3 sentences.
   **Category**: Categorize the article into one or more retail-focused categories:["product_innovation", "pricing_strategy", "market_expansion", "retail_tech", "sustainability", "supply_chain_risk", "competitor_moves", "consumer_trends", "others"].
   **Entity Extraction**: Extract and identify brands/compagnies,  products, regions/markets, Financial Data, dates
   **sentiments**:Analyze the overall sentiment: Positive, Negative, or Neutral, Market Sentiment, Competitive Sentiment, Supply Chain Sentiment. Provide a brief justification.
   **trend detection **:Identify whether the article includes any of the following:Product launches or innovations, Pricing or promotion strategies, Store openings, expansions, or mergers, Retail technology integrations, Sustainability initiatives, Supply chain risks: delays, shortages, geopolitical events
   **predictive intelligence **:Predict whether the article suggests:Short-term Implications, Medium-term Trends, Long-term Forecasts, Recommendation Priority, An emerging trend, A supply chain risk, A market shift, Or none
   **Business Impact Scoring**: Rate the potential impact of the article on the business on a scale of 1 to 10, where 1 is low impact and 10 is high impact. Provide a brief justification.
   
   
   """
     for i, article in enumerate(articles)]
)

# Create prompt
prompt_template = f"""
You are an expert retail intelligence analyst working for a major retail/FMCG company that serves 15,000 suppliers and 200 retailers. Your role is to analyze news articles and extract actionable business intelligence that will help stakeholders make informed decisions about competitive positioning, supply chain management, and market opportunities.

## ROLE
Act as a Senior Retail Intelligence Analyst with expertise in:
- Competitive intelligence and market analysis
- Supply chain risk assessment
- Predictive trend analysis
- FMCG/Retail industry dynamics
- Business impact evaluation

## ACTION
Analyze the provided news article and perform the following comprehensive analysis:

### 1. INTELLIGENT CLASSIFICATION
Categorize the article into ONE primary category:
- **product_innovation**: New products, launches, and innovations
- **pricing_strategy**: Pricing strategies, promotions, and competitive pricing
- **market_expansion**: Store openings, market expansion, and geographic growth
- **supply_chain**: Supply chain disruptions, logistics, and sourcing issues
- **technology_retail**: Retail technology, digital transformation, and e-commerce
- **sustainability**: Sustainability initiatives and ESG practices
- **consumer_behavior**: Consumer trends, preferences, and behavioral shifts
- **regulatory_compliance**: Regulatory changes, compliance, and policy impacts
- **competitive_intelligence**: Competitor moves, market share, and strategic partnerships
- **risk_management**: Geopolitical risks, economic factors, and crisis management

### 2. ENTITY EXTRACTION
Extract and identify:
- **Brands/Companies**: All mentioned retail brands, suppliers, competitors
- **Products**: Specific products, categories, or product lines
- **Regions/Markets**: Geographic locations, countries, cities, markets
- **Key Figures**: Important people, executives, industry leaders
- **Financial Data**: Revenue, sales figures, market share, investments
- **Dates**: Important dates, timelines, deadlines

### 3. RETAIL-SPECIFIC SENTIMENT ANALYSIS
Assess sentiment across three dimensions:
- **Market Sentiment**: Positive/Negative/Neutral impact on market conditions
- **Competitive Sentiment**: Advantage/Disadvantage/Neutral for competitive positioning
- **Supply Chain Sentiment**: Stable/Risk/Opportunity for supply chain operations

### 4. TREND DETECTION & INNOVATION ANALYSIS
Identify:
- **Emerging Trends**: New consumer behaviors, market shifts, technological adoption
- **Innovation Signals**: R&D developments, patent filings, breakthrough technologies
- **Product Launches**: New product introductions, line extensions, category expansions
- **Market Disruptions**: Business model changes, new market entrants, industry shifts

### 5. SUPPLY CHAIN RISK ASSESSMENT
Evaluate:
- **Disruption Risks**: Logistics issues, transportation problems, warehouse challenges
- **Supplier Issues**: Supplier bankruptcies, quality problems, capacity constraints
- **Geopolitical Risks**: Trade tensions, sanctions, regulatory changes
- **Economic Factors**: Inflation, currency fluctuations, commodity price changes

### 6. PREDICTIVE INTELLIGENCE
Based on the information, predict:
- **Short-term Implications** (1-3 months): Immediate business impacts
- **Medium-term Trends** (3-12 months): Evolving market dynamics
- **Long-term Forecasts** (1-3 years): Strategic industry shifts
- **Recommendation Priority**: High/Medium/Low urgency for stakeholder action

### 7. BUSINESS IMPACT SCORING
Rate the criticality on a scale of 1-10:
- **Competitive Impact**: How significantly this affects competitive positioning
- **Supply Chain Impact**: Risk level for supply chain operations
- **Market Opportunity**: Potential for new business opportunities
- **Strategic Importance**: Overall relevance to business strategy


## FORMAT

### Here's how I'd like you to return my results:

Dear Customer,

Welcome to our weekly newsletter, where we provide you with a brief overview summary of significant events. 

### Article Summaries:


{article_summaries}

Stay tuned for our next issue as we delve deeper into the evolving challenges and opportunities shaping the retail landscape.

Warm regards,
Your retail.ai team



##There are several aspects we look for in a good summary:
 **Consistency**: A good summary should be consistent with the article. Please do not add any new information to the summary, even if you know more about the article's content. For example, if the article does mention the first name of 'President Biden', please do not use the full name 'Joe Biden' in your summary.
 **Relevance**: A good summary should only include important information from the article. In general, you should include the key points you highlighted in your summary, but feel free to omit details you think are unnecessary to convey the main idea of the article.
 **Conciseness**: A good summary should be short and to the point. But it is okay if the summary is shorter when the article is short.
 **Coherence**: A good summary is more than just a list of sentences. It should be a coherent paragraph that makes sense.


## TONE
Maintain a professional, analytical, and strategic tone throughout your analysis. Be:
- **Objective**: Base conclusions on factual evidence from the text
- **Actionable**: Provide insights that can drive business decisions
- **Comprehensive**: Cover all relevant aspects without being verbose
- **Strategic**: Focus on business implications and competitive advantages
- **Precise**: Use specific retail/FMCG terminology and metrics


## CONSTRAINTS
- Only analyze information explicitly mentioned in the provided article
- If information is not available for a section, use "null" or empty arrays
- Maintain consistency in entity naming (e.g., "Walmart" not "Wal-Mart")
- Prioritize actionable insights over general observations




### Input Articles:
```{articles_text}``` 
"""
###################

# Measuring latency
start_time = time.time()

################
# Générer la réponse
response = client.chat.completions.create(
    model="gpt-4o-mini-2024-07-18",
    messages=[{"role":"user","content":prompt_template}],
    #prompt=prompt_template,
    max_tokens=8096,
    top_p=0.5,
    temperature=0.3
)

#######################
# Calculate latency and total processing time
latency = time.time() - start_time
#print(chat_completion.choices[0].message.content)
#output_text = chat_completion.choices[0].message.content
output_text = response.choices[0].message.content

# Number of tokens generated in the response
output_tokens = len(output_text.split())


# Calculate tokens for input and output
input_tokens = tokenizer(prompt_template)['input_ids']
output_tokens = tokenizer(output_text)['input_ids']

# Calculate the number of tokens
num_input_tokens = len(input_tokens)
num_output_tokens = len(output_tokens)

# Calculate word length for input and output
input_word_count = len(prompt_template.split())
output_word_count = len(output_text.split())


# Price calculation
# Price in USD per million input and output  tokens
price_per_million_input_tokens = 0.15  
price_per_million_output_tokens = 0.60  


# Calculation of entry and exit costs
input_cost = (num_input_tokens / 1_000_000) * price_per_million_input_tokens
output_cost = (num_output_tokens / 1_000_000) * price_per_million_output_tokens
total_cost = input_cost + output_cost


# Define the folder path and create folders if necessary
output_dir = os.path.join("Output", "Ergebnis_test")
os.makedirs(output_dir, exist_ok=True)

# Define the file name with the current date and time
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
file_name = f"{current_time}_zs_gpt4o_mini.txt"
file_path = os.path.join(output_dir, file_name)

# Save the result in the text file
with open(file_path, "w") as file:
    file.write(output_text)
    

#print(f"Output saved to {file_path}")
print(f"Output saved to {file_path}")


# Function to save the analysis in an Excel file

def save_analysis_to_excel(model_name, context_window, output_speed, latency, price, output_price, input_price, time_period, num_input_tokens, num_output_tokens, input_word_count, output_word_count, base_dir="LLM_Analysis"):
    data = {
        "Context Window": [context_window],
        "Output Speed (tokens/s)": [output_speed],
        "Latency (seconds)": [latency],
        "Total Cost ($)": [price],
        "Output Price ($ per M tokens)": [output_price],
        "Input Price ($ per M tokens)": [input_price],
        "Time Period (seconds)": [time_period],
        "Input Tokens": [num_input_tokens],
        "Output Tokens": [num_output_tokens],
        "Input Word Count": [input_word_count],
        "Output Word Count": [output_word_count]
    }
    
    df = pd.DataFrame(data)
    # Create the path to the Excel file
    output_dir = os.path.join(base_dir, model_name)
    os.makedirs(output_dir, exist_ok=True)
    file_name = f"{model_name}_analysis_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.xlsx"
    file_path = os.path.join(output_dir, file_name)
    try:
        df.to_excel(file_path, sheet_name=model_name[:31], index=False)
    except Exception as e:
        print(f"An error occurred: {e}")
           
        
            

# An example of use
context_window = 1047576  # Example of the size of the context window
output_speed = num_output_tokens / latency  # Tokens per second
time_period = latency  # Total time in seconds

model_name = f"zs_gpt4o_mini{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
save_analysis_to_excel(
    model_name=model_name,
    context_window=context_window,
    output_speed=output_speed,
    latency=latency,
    price=total_cost,
    output_price=price_per_million_output_tokens,
    input_price=price_per_million_input_tokens,
    time_period=time_period,
    num_input_tokens=num_input_tokens,
    num_output_tokens=num_output_tokens,
    input_word_count=input_word_count,
    output_word_count=output_word_count
)





2025-07-06 16:50:13,454 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Output saved to Output\Ergebnis_test\2025-07-06_16-50-13_zs_gpt4o_mini.txt


## OpenAI (gpt41Mini)

In [42]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4.1-nano-2025-04-14",
    messages=[
        {
            "role": "user",
            "content": "Write a one-sentence bedtime story about a unicorn."
        }
    ]
)

print(completion.choices[0].message.content)

2025-07-06 16:35:39,159 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Once upon a time, a gentle unicorn with sparkling hooves and a rainbow mane found a cozy cloud to sleep on beneath the twinkling stars.


In [ ]:
def get_articles_with_metadata(df, num_articles=5, offset=0):
    articles = df.iloc[offset:offset + num_articles][['url',
 'title',
 'publish_date',
 'authors',
 'scraped_at',
 'keywords',
 'text',
 'text_length']].to_dict(orient='records')
    return articles

# Example to obtain the first 5 articles
articles = get_articles_with_metadata(df, num_articles=5, offset=0)

# Using metadata to build article text
articles_text = "\n\n".join(
    [f"### Article {i+1}:\nTitle: {article['title']}\nURL: {article['url']}\nPublish_date: {article['publish_date']}\n"
     for i, article in enumerate(articles)]
)

# Dynamically generate the ‘Article Summaries’ section with numbers
article_summaries = "\n\n".join(
    [f"""### {i+1}. 
     
     **Title**: {article['title']}
   **Link to source**: {article['url']} 
   **Publish_date**: {article['publish_date']}
   **Summary**: Summarize the key points of the article in 2-3 sentences.
   **Category**: Categorize the article into one or more retail-focused categories:["product_innovation", "pricing_strategy", "market_expansion", "retail_tech", "sustainability", "supply_chain_risk", "competitor_moves", "consumer_trends", "others"].
   **Entity Extraction**: Extract and identify brands/compagnies,  products, regions/markets, Financial Data, dates
   **sentiments**:Analyze the overall sentiment: Positive, Negative, or Neutral, Market Sentiment, Competitive Sentiment, Supply Chain Sentiment. Provide a brief justification.
   **trend detection **:Identify whether the article includes any of the following:Product launches or innovations, Pricing or promotion strategies, Store openings, expansions, or mergers, Retail technology integrations, Sustainability initiatives, Supply chain risks: delays, shortages, geopolitical events
   **predictive intelligence **:Predict whether the article suggests:Short-term Implications, Medium-term Trends, Long-term Forecasts, Recommendation Priority, An emerging trend, A supply chain risk, A market shift, Or none
   **Business Impact Scoring**: Rate the potential impact of the article on the business on a scale of 1 to 10, where 1 is low impact and 10 is high impact. Provide a brief justification.
   
   
   """
     for i, article in enumerate(articles)]
)

# Create prompt
prompt_template = f"""
You are an expert retail intelligence analyst working for a major retail/FMCG company that serves 15,000 suppliers and 200 retailers. Your role is to analyze news articles and extract actionable business intelligence that will help stakeholders make informed decisions about competitive positioning, supply chain management, and market opportunities.

## ROLE
Act as a Senior Retail Intelligence Analyst with expertise in:
- Competitive intelligence and market analysis
- Supply chain risk assessment
- Predictive trend analysis
- FMCG/Retail industry dynamics
- Business impact evaluation

## ACTION
Analyze the provided news article and perform the following comprehensive analysis:

### 1. INTELLIGENT CLASSIFICATION
Categorize the article into ONE primary category:
- **product_innovation**: New products, launches, and innovations
- **pricing_strategy**: Pricing strategies, promotions, and competitive pricing
- **market_expansion**: Store openings, market expansion, and geographic growth
- **supply_chain**: Supply chain disruptions, logistics, and sourcing issues
- **technology_retail**: Retail technology, digital transformation, and e-commerce
- **sustainability**: Sustainability initiatives and ESG practices
- **consumer_behavior**: Consumer trends, preferences, and behavioral shifts
- **regulatory_compliance**: Regulatory changes, compliance, and policy impacts
- **competitive_intelligence**: Competitor moves, market share, and strategic partnerships
- **risk_management**: Geopolitical risks, economic factors, and crisis management

### 2. ENTITY EXTRACTION
Extract and identify:
- **Brands/Companies**: All mentioned retail brands, suppliers, competitors
- **Products**: Specific products, categories, or product lines
- **Regions/Markets**: Geographic locations, countries, cities, markets
- **Key Figures**: Important people, executives, industry leaders
- **Financial Data**: Revenue, sales figures, market share, investments
- **Dates**: Important dates, timelines, deadlines

### 3. RETAIL-SPECIFIC SENTIMENT ANALYSIS
Assess sentiment across three dimensions:
- **Market Sentiment**: Positive/Negative/Neutral impact on market conditions
- **Competitive Sentiment**: Advantage/Disadvantage/Neutral for competitive positioning
- **Supply Chain Sentiment**: Stable/Risk/Opportunity for supply chain operations

### 4. TREND DETECTION & INNOVATION ANALYSIS
Identify:
- **Emerging Trends**: New consumer behaviors, market shifts, technological adoption
- **Innovation Signals**: R&D developments, patent filings, breakthrough technologies
- **Product Launches**: New product introductions, line extensions, category expansions
- **Market Disruptions**: Business model changes, new market entrants, industry shifts

### 5. SUPPLY CHAIN RISK ASSESSMENT
Evaluate:
- **Disruption Risks**: Logistics issues, transportation problems, warehouse challenges
- **Supplier Issues**: Supplier bankruptcies, quality problems, capacity constraints
- **Geopolitical Risks**: Trade tensions, sanctions, regulatory changes
- **Economic Factors**: Inflation, currency fluctuations, commodity price changes

### 6. PREDICTIVE INTELLIGENCE
Based on the information, predict:
- **Short-term Implications** (1-3 months): Immediate business impacts
- **Medium-term Trends** (3-12 months): Evolving market dynamics
- **Long-term Forecasts** (1-3 years): Strategic industry shifts
- **Recommendation Priority**: High/Medium/Low urgency for stakeholder action

### 7. BUSINESS IMPACT SCORING
Rate the criticality on a scale of 1-10:
- **Competitive Impact**: How significantly this affects competitive positioning
- **Supply Chain Impact**: Risk level for supply chain operations
- **Market Opportunity**: Potential for new business opportunities
- **Strategic Importance**: Overall relevance to business strategy


## FORMAT

### Here's how I'd like you to return my results:

Dear Customer,

Welcome to our weekly newsletter, where we provide you with a brief overview summary of significant events. 

### Article Summaries:


{article_summaries}

Stay tuned for our next issue as we delve deeper into the evolving challenges and opportunities shaping the retail landscape.

Warm regards,
Your retail.ai team



##There are several aspects we look for in a good summary:
 **Consistency**: A good summary should be consistent with the article. Please do not add any new information to the summary, even if you know more about the article's content. For example, if the article does mention the first name of 'President Biden', please do not use the full name 'Joe Biden' in your summary.
 **Relevance**: A good summary should only include important information from the article. In general, you should include the key points you highlighted in your summary, but feel free to omit details you think are unnecessary to convey the main idea of the article.
 **Conciseness**: A good summary should be short and to the point. But it is okay if the summary is shorter when the article is short.
 **Coherence**: A good summary is more than just a list of sentences. It should be a coherent paragraph that makes sense.


## TONE
Maintain a professional, analytical, and strategic tone throughout your analysis. Be:
- **Objective**: Base conclusions on factual evidence from the text
- **Actionable**: Provide insights that can drive business decisions
- **Comprehensive**: Cover all relevant aspects without being verbose
- **Strategic**: Focus on business implications and competitive advantages
- **Precise**: Use specific retail/FMCG terminology and metrics


## CONSTRAINTS
- Only analyze information explicitly mentioned in the provided article
- If information is not available for a section, use "null" or empty arrays
- Maintain consistency in entity naming (e.g., "Walmart" not "Wal-Mart")
- Prioritize actionable insights over general observations




### Input Articles:
```{articles_text}``` 
"""
###################

# Measuring latency
start_time = time.time()

################
# Générer la réponse
response = client.chat.completions.create(
    model="gpt-4.1-mini-2025-04-14",
    messages=[{"role":"user","content":prompt_template}],
    #prompt=prompt_template,
    max_tokens=8096,
    top_p=0.5,
    temperature=0.3
)

#######################
# Calculate latency and total processing time
latency = time.time() - start_time
#print(chat_completion.choices[0].message.content)
#output_text = chat_completion.choices[0].message.content
output_text = response.choices[0].message.content

# Number of tokens generated in the response
output_tokens = len(output_text.split())

# Calculate tokens for input and output
input_tokens = tokenizer(prompt_template)['input_ids']
output_tokens = tokenizer(output_text)['input_ids']

# Calculate the number of tokens
num_input_tokens = len(input_tokens)
num_output_tokens = len(output_tokens)

# Calculate word length for input and output
input_word_count = len(prompt_template.split())
output_word_count = len(output_text.split())


# Price calculation

# Price in USD per million input and output  tokens
price_per_million_input_tokens = 0.40  
price_per_million_output_tokens = 1.60  


# Calculation of entry and exit costs
input_cost = (num_input_tokens / 1_000_000) * price_per_million_input_tokens
output_cost = (num_output_tokens / 1_000_000) * price_per_million_output_tokens
total_cost = input_cost + output_cost


# Define the folder path and create folders if necessary
output_dir = os.path.join("Output", "Ergebnis_test")
os.makedirs(output_dir, exist_ok=True)

# Define the file name with the current date and time
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
file_name = f"{current_time}_zs_gpt41_mini.txt"
file_path = os.path.join(output_dir, file_name)

# Save the result in the text file
with open(file_path, "w") as file:
    file.write(output_text)
    

#print(f"Output saved to {file_path}")
print(f"Output saved to {file_path}")


# Function to save the analysis in an Excel file

def save_analysis_to_excel(model_name, context_window, output_speed, latency, price, output_price, input_price, time_period, num_input_tokens, num_output_tokens, input_word_count, output_word_count, base_dir="LLM_Analysis"):
    data = {
        "Context Window": [context_window],
        "Output Speed (tokens/s)": [output_speed],
        "Latency (seconds)": [latency],
        "Total Cost ($)": [price],
        "Output Price ($ per M tokens)": [output_price],
        "Input Price ($ per M tokens)": [input_price],
        "Time Period (seconds)": [time_period],
        "Input Tokens": [num_input_tokens],
        "Output Tokens": [num_output_tokens],
        "Input Word Count": [input_word_count],
        "Output Word Count": [output_word_count]
    }
    
    df = pd.DataFrame(data)
   # Create the path to the Excel file
    output_dir = os.path.join(base_dir, model_name)
    os.makedirs(output_dir, exist_ok=True)
    file_name = f"{model_name}_analysis_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.xlsx"
    file_path = os.path.join(output_dir, file_name)
    try:
        df.to_excel(file_path, sheet_name=model_name[:31], index=False)
    except Exception as e:
        print(f"An error occurred: {e}")
           
        
            

# An example of use
context_window = 1047576  # Example of the size of the context window
output_speed = num_output_tokens / latency  # Tokens per second
time_period = latency  # Total time in seconds

model_name = f"zs_gpt41_mini{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
save_analysis_to_excel(
    model_name=model_name,
    context_window=context_window,
    output_speed=output_speed,
    latency=latency,
    price=total_cost,
    output_price=price_per_million_output_tokens,
    input_price=price_per_million_input_tokens,
    time_period=time_period,
    num_input_tokens=num_input_tokens,
    num_output_tokens=num_output_tokens,
    input_word_count=input_word_count,
    output_word_count=output_word_count
)





2025-07-06 16:45:10,700 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Output saved to Output\Ergebnis_test\2025-07-06_16-45-10_zs_gpt41_mini.txt


## meta-llama/llama-4-maverick

In [ ]:
def get_articles_with_metadata(df, num_articles=5, offset=0):
    articles = df.iloc[offset:offset + num_articles][['url',
 'title',
 'publish_date',
 'authors',
 'scraped_at',
 'keywords',
 'text',
 'text_length']].to_dict(orient='records')
    return articles

# Example to obtain the first 10 articles
articles = get_articles_with_metadata(df, num_articles=5, offset=0)

# Using metadata to build article text
articles_text = "\n\n".join(
    [f"### Article {i+1}:\nTitle: {article['title']}\nURL: {article['url']}\nPublish_date: {article['publish_date']}\n"
     for i, article in enumerate(articles)]
)

# Dynamically generate the ‘Article Summaries’ section with numbers
article_summaries = "\n\n".join(
    [f"""### {i+1}. 
     
     **Title**: {article['title']}
   **Link to source**: {article['url']} 
   **Publish_date**: {article['publish_date']}
   **Summary**: Summarize the key points of the article in 2-3 sentences.
   **Category**: Categorize the article into one or more retail-focused categories:["product_innovation", "pricing_strategy", "market_expansion", "retail_tech", "sustainability", "supply_chain_risk", "competitor_moves", "consumer_trends", "others"].
   **Entity Extraction**: Extract and identify brands/compagnies,  products, regions/markets, Financial Data, dates
   **sentiments**:Analyze the overall sentiment: Positive, Negative, or Neutral, Market Sentiment, Competitive Sentiment, Supply Chain Sentiment. Provide a brief justification.
   **trend detection **:Identify whether the article includes any of the following:Product launches or innovations, Pricing or promotion strategies, Store openings, expansions, or mergers, Retail technology integrations, Sustainability initiatives, Supply chain risks: delays, shortages, geopolitical events
   **predictive intelligence **:Predict whether the article suggests:Short-term Implications, Medium-term Trends, Long-term Forecasts, Recommendation Priority, An emerging trend, A supply chain risk, A market shift, Or none
   **Business Impact Scoring**: Rate the potential impact of the article on the business on a scale of 1 to 10, where 1 is low impact and 10 is high impact. Provide a brief justification.
   
   
   """
     for i, article in enumerate(articles)]
)

# Create prompt
prompt_template = f"""
You are an expert retail intelligence analyst working for a major retail/FMCG company that serves 15,000 suppliers and 200 retailers. Your role is to analyze news articles and extract actionable business intelligence that will help stakeholders make informed decisions about competitive positioning, supply chain management, and market opportunities.

## ROLE
Act as a Senior Retail Intelligence Analyst with expertise in:
- Competitive intelligence and market analysis
- Supply chain risk assessment
- Predictive trend analysis
- FMCG/Retail industry dynamics
- Business impact evaluation

## ACTION
Analyze the provided news article and perform the following comprehensive analysis:

### 1. INTELLIGENT CLASSIFICATION
Categorize the article into ONE primary category:
- **product_innovation**: New products, launches, and innovations
- **pricing_strategy**: Pricing strategies, promotions, and competitive pricing
- **market_expansion**: Store openings, market expansion, and geographic growth
- **supply_chain**: Supply chain disruptions, logistics, and sourcing issues
- **technology_retail**: Retail technology, digital transformation, and e-commerce
- **sustainability**: Sustainability initiatives and ESG practices
- **consumer_behavior**: Consumer trends, preferences, and behavioral shifts
- **regulatory_compliance**: Regulatory changes, compliance, and policy impacts
- **competitive_intelligence**: Competitor moves, market share, and strategic partnerships
- **risk_management**: Geopolitical risks, economic factors, and crisis management

### 2. ENTITY EXTRACTION
Extract and identify:
- **Brands/Companies**: All mentioned retail brands, suppliers, competitors
- **Products**: Specific products, categories, or product lines
- **Regions/Markets**: Geographic locations, countries, cities, markets
- **Key Figures**: Important people, executives, industry leaders
- **Financial Data**: Revenue, sales figures, market share, investments
- **Dates**: Important dates, timelines, deadlines

### 3. RETAIL-SPECIFIC SENTIMENT ANALYSIS
Assess sentiment across three dimensions:
- **Market Sentiment**: Positive/Negative/Neutral impact on market conditions
- **Competitive Sentiment**: Advantage/Disadvantage/Neutral for competitive positioning
- **Supply Chain Sentiment**: Stable/Risk/Opportunity for supply chain operations

### 4. TREND DETECTION & INNOVATION ANALYSIS
Identify:
- **Emerging Trends**: New consumer behaviors, market shifts, technological adoption
- **Innovation Signals**: R&D developments, patent filings, breakthrough technologies
- **Product Launches**: New product introductions, line extensions, category expansions
- **Market Disruptions**: Business model changes, new market entrants, industry shifts

### 5. SUPPLY CHAIN RISK ASSESSMENT
Evaluate:
- **Disruption Risks**: Logistics issues, transportation problems, warehouse challenges
- **Supplier Issues**: Supplier bankruptcies, quality problems, capacity constraints
- **Geopolitical Risks**: Trade tensions, sanctions, regulatory changes
- **Economic Factors**: Inflation, currency fluctuations, commodity price changes

### 6. PREDICTIVE INTELLIGENCE
Based on the information, predict:
- **Short-term Implications** (1-3 months): Immediate business impacts
- **Medium-term Trends** (3-12 months): Evolving market dynamics
- **Long-term Forecasts** (1-3 years): Strategic industry shifts
- **Recommendation Priority**: High/Medium/Low urgency for stakeholder action

### 7. BUSINESS IMPACT SCORING
Rate the criticality on a scale of 1-10:
- **Competitive Impact**: How significantly this affects competitive positioning
- **Supply Chain Impact**: Risk level for supply chain operations
- **Market Opportunity**: Potential for new business opportunities
- **Strategic Importance**: Overall relevance to business strategy


## FORMAT

### Here's how I'd like you to return my results:

Dear Customer,

Welcome to our weekly newsletter, where we provide you with a brief overview summary of significant events. 

### Article Summaries:


{article_summaries}

Stay tuned for our next issue as we delve deeper into the evolving challenges and opportunities shaping the retail landscape.

Warm regards,
Your retail.ai team



##There are several aspects we look for in a good summary:
 **Consistency**: A good summary should be consistent with the article. Please do not add any new information to the summary, even if you know more about the article's content. For example, if the article does mention the first name of 'President Biden', please do not use the full name 'Joe Biden' in your summary.
 **Relevance**: A good summary should only include important information from the article. In general, you should include the key points you highlighted in your summary, but feel free to omit details you think are unnecessary to convey the main idea of the article.
 **Conciseness**: A good summary should be short and to the point. But it is okay if the summary is shorter when the article is short.
 **Coherence**: A good summary is more than just a list of sentences. It should be a coherent paragraph that makes sense.


## TONE
Maintain a professional, analytical, and strategic tone throughout your analysis. Be:
- **Objective**: Base conclusions on factual evidence from the text
- **Actionable**: Provide insights that can drive business decisions
- **Comprehensive**: Cover all relevant aspects without being verbose
- **Strategic**: Focus on business implications and competitive advantages
- **Precise**: Use specific retail/FMCG terminology and metrics


## CONSTRAINTS
- Only analyze information explicitly mentioned in the provided article
- If information is not available for a section, use "null" or empty arrays
- Maintain consistency in entity naming (e.g., "Walmart" not "Wal-Mart")
- Prioritize actionable insights over general observations




### Input Articles:
```{articles_text}``` 
"""
###################

# Measuring latency
start_time = time.time()

################
chat_completion = client.chat.completions.create(
    model="meta-llama/llama-4-maverick-17b-128e-instruct",
    messages=[{"role":"user","content":prompt_template}],
    #prompt=prompt_template,
    max_tokens=8096,
    top_p=0.5,
    temperature=0.3
)

#######################
# Calculate latency and total processing time
latency = time.time() - start_time
#print(chat_completion.choices[0].message.content)
output_text = chat_completion.choices[0].message.content


# Number of tokens generated in the response
output_tokens = len(output_text.split())


# Calculate tokens for input and output
input_tokens = tokenizer(prompt_template)['input_ids']
output_tokens = tokenizer(output_text)['input_ids']

# Calculate the number of tokens
num_input_tokens = len(input_tokens)
num_output_tokens = len(output_tokens)

# Calculate word length for input and output
input_word_count = len(prompt_template.split())
output_word_count = len(output_text.split())


# Price calculation

# Price in USD per million input and output  tokens
price_per_million_input_tokens = 0.20  
price_per_million_output_tokens = 0.60  


# Calculation of entry and exit costs
input_cost = (num_input_tokens / 1_000_000) * price_per_million_input_tokens
output_cost = (num_output_tokens / 1_000_000) * price_per_million_output_tokens
total_cost = input_cost + output_cost


# Define the folder path and create folders if necessary
output_dir = os.path.join("Output", "Ergebnis_test")
os.makedirs(output_dir, exist_ok=True)

# Define the file name with the current date and time
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
file_name = f"{current_time}_zs_llama4.txt"
file_path = os.path.join(output_dir, file_name)

# Save the result in the text file
with open(file_path, "w") as file:
    file.write(output_text)
    

#print(f"Output saved to {file_path}")
print(f"Output saved to {file_path}")


# Function to save the analysis in an Excel file

def save_analysis_to_excel(model_name, context_window, output_speed, latency, price, output_price, input_price, time_period, num_input_tokens, num_output_tokens, input_word_count, output_word_count, base_dir="LLM_Analysis"):
    data = {
        "Context Window": [context_window],
        "Output Speed (tokens/s)": [output_speed],
        "Latency (seconds)": [latency],
        "Total Cost ($)": [price],
        "Output Price ($ per M tokens)": [output_price],
        "Input Price ($ per M tokens)": [input_price],
        "Time Period (seconds)": [time_period],
        "Input Tokens": [num_input_tokens],
        "Output Tokens": [num_output_tokens],
        "Input Word Count": [input_word_count],
        "Output Word Count": [output_word_count]
    }
    
    df = pd.DataFrame(data)
   # Create the path to the Excel file
    output_dir = os.path.join(base_dir, model_name)
    os.makedirs(output_dir, exist_ok=True)
    file_name = f"{model_name}_analysis_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.xlsx"
    file_path = os.path.join(output_dir, file_name)
    try:
        df.to_excel(file_path, sheet_name=model_name[:31], index=False)
    except Exception as e:
        print(f"An error occurred: {e}")
           
        
            

# An example of use
context_window = 131072  #Example of the size of the context window
output_speed = num_output_tokens / latency  # Tokens per second
time_period = latency  # Total time in seconds

model_name = f"zs_llama4{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
save_analysis_to_excel(
    model_name=model_name,
    context_window=context_window,
    output_speed=output_speed,
    latency=latency,
    price=total_cost,
    output_price=price_per_million_output_tokens,
    input_price=price_per_million_input_tokens,
    time_period=time_period,
    num_input_tokens=num_input_tokens,
    num_output_tokens=num_output_tokens,
    input_word_count=input_word_count,
    output_word_count=output_word_count
)





## deepseek-r1-distill

In [ ]:
def get_articles_with_metadata(df, num_articles=5, offset=0):
    articles = df.iloc[offset:offset + num_articles][['url',
 'title',
 'publish_date',
 'authors',
 'scraped_at',
 'keywords',
 'text',
 'text_length']].to_dict(orient='records')
    return articles

# Example to obtain the first 5 articles
articles = get_articles_with_metadata(df, num_articles=5, offset=0)

# Using metadata to build article text
articles_text = "\n\n".join(
    [f"### Article {i+1}:\nTitle: {article['title']}\nURL: {article['url']}\nPublish_date: {article['publish_date']}\n"
     for i, article in enumerate(articles)]
)

# Dynamically generate the ‘Article Summaries’ section with numbers
article_summaries = "\n\n".join(
    [f"""### {i+1}. 
     
     **Title**: {article['title']}
   **Link to source**: {article['url']} 
   **Publish_date**: {article['publish_date']}
   **Summary**: Summarize the key points of the article in 2-3 sentences.
   **Category**: Categorize the article into one or more retail-focused categories:["product_innovation", "pricing_strategy", "market_expansion", "retail_tech", "sustainability", "supply_chain_risk", "competitor_moves", "consumer_trends", "others"].
   **Entity Extraction**: Extract and identify brands/compagnies,  products, regions/markets, Financial Data, dates
   **sentiments**:Analyze the overall sentiment: Positive, Negative, or Neutral, Market Sentiment, Competitive Sentiment, Supply Chain Sentiment. Provide a brief justification.
   **trend detection **:Identify whether the article includes any of the following:Product launches or innovations, Pricing or promotion strategies, Store openings, expansions, or mergers, Retail technology integrations, Sustainability initiatives, Supply chain risks: delays, shortages, geopolitical events
   **predictive intelligence **:Predict whether the article suggests:Short-term Implications, Medium-term Trends, Long-term Forecasts, Recommendation Priority, An emerging trend, A supply chain risk, A market shift, Or none
   **Business Impact Scoring**: Rate the potential impact of the article on the business on a scale of 1 to 10, where 1 is low impact and 10 is high impact. Provide a brief justification.
   
   
   """
     for i, article in enumerate(articles)]
)

# Create prompt
prompt_template = f"""
You are an expert retail intelligence analyst working for a major retail/FMCG company that serves 15,000 suppliers and 200 retailers. Your role is to analyze news articles and extract actionable business intelligence that will help stakeholders make informed decisions about competitive positioning, supply chain management, and market opportunities.

## ROLE
Act as a Senior Retail Intelligence Analyst with expertise in:
- Competitive intelligence and market analysis
- Supply chain risk assessment
- Predictive trend analysis
- FMCG/Retail industry dynamics
- Business impact evaluation

## ACTION
Analyze the provided news article and perform the following comprehensive analysis:

### 1. INTELLIGENT CLASSIFICATION
Categorize the article into ONE primary category:
- **product_innovation**: New products, launches, and innovations
- **pricing_strategy**: Pricing strategies, promotions, and competitive pricing
- **market_expansion**: Store openings, market expansion, and geographic growth
- **supply_chain**: Supply chain disruptions, logistics, and sourcing issues
- **technology_retail**: Retail technology, digital transformation, and e-commerce
- **sustainability**: Sustainability initiatives and ESG practices
- **consumer_behavior**: Consumer trends, preferences, and behavioral shifts
- **regulatory_compliance**: Regulatory changes, compliance, and policy impacts
- **competitive_intelligence**: Competitor moves, market share, and strategic partnerships
- **risk_management**: Geopolitical risks, economic factors, and crisis management

### 2. ENTITY EXTRACTION
Extract and identify:
- **Brands/Companies**: All mentioned retail brands, suppliers, competitors
- **Products**: Specific products, categories, or product lines
- **Regions/Markets**: Geographic locations, countries, cities, markets
- **Key Figures**: Important people, executives, industry leaders
- **Financial Data**: Revenue, sales figures, market share, investments
- **Dates**: Important dates, timelines, deadlines

### 3. RETAIL-SPECIFIC SENTIMENT ANALYSIS
Assess sentiment across three dimensions:
- **Market Sentiment**: Positive/Negative/Neutral impact on market conditions
- **Competitive Sentiment**: Advantage/Disadvantage/Neutral for competitive positioning
- **Supply Chain Sentiment**: Stable/Risk/Opportunity for supply chain operations

### 4. TREND DETECTION & INNOVATION ANALYSIS
Identify:
- **Emerging Trends**: New consumer behaviors, market shifts, technological adoption
- **Innovation Signals**: R&D developments, patent filings, breakthrough technologies
- **Product Launches**: New product introductions, line extensions, category expansions
- **Market Disruptions**: Business model changes, new market entrants, industry shifts

### 5. SUPPLY CHAIN RISK ASSESSMENT
Evaluate:
- **Disruption Risks**: Logistics issues, transportation problems, warehouse challenges
- **Supplier Issues**: Supplier bankruptcies, quality problems, capacity constraints
- **Geopolitical Risks**: Trade tensions, sanctions, regulatory changes
- **Economic Factors**: Inflation, currency fluctuations, commodity price changes

### 6. PREDICTIVE INTELLIGENCE
Based on the information, predict:
- **Short-term Implications** (1-3 months): Immediate business impacts
- **Medium-term Trends** (3-12 months): Evolving market dynamics
- **Long-term Forecasts** (1-3 years): Strategic industry shifts
- **Recommendation Priority**: High/Medium/Low urgency for stakeholder action

### 7. BUSINESS IMPACT SCORING
Rate the criticality on a scale of 1-10:
- **Competitive Impact**: How significantly this affects competitive positioning
- **Supply Chain Impact**: Risk level for supply chain operations
- **Market Opportunity**: Potential for new business opportunities
- **Strategic Importance**: Overall relevance to business strategy


## FORMAT

### Here's how I'd like you to return my results:

Dear Customer,

Welcome to our weekly newsletter, where we provide you with a brief overview summary of significant events. 

### Article Summaries:


{article_summaries}

Stay tuned for our next issue as we delve deeper into the evolving challenges and opportunities shaping the retail landscape.

Warm regards,
Your retail.ai team



##There are several aspects we look for in a good summary:
 **Consistency**: A good summary should be consistent with the article. Please do not add any new information to the summary, even if you know more about the article's content. For example, if the article does mention the first name of 'President Biden', please do not use the full name 'Joe Biden' in your summary.
 **Relevance**: A good summary should only include important information from the article. In general, you should include the key points you highlighted in your summary, but feel free to omit details you think are unnecessary to convey the main idea of the article.
 **Conciseness**: A good summary should be short and to the point. But it is okay if the summary is shorter when the article is short.
 **Coherence**: A good summary is more than just a list of sentences. It should be a coherent paragraph that makes sense.


## TONE
Maintain a professional, analytical, and strategic tone throughout your analysis. Be:
- **Objective**: Base conclusions on factual evidence from the text
- **Actionable**: Provide insights that can drive business decisions
- **Comprehensive**: Cover all relevant aspects without being verbose
- **Strategic**: Focus on business implications and competitive advantages
- **Precise**: Use specific retail/FMCG terminology and metrics


## CONSTRAINTS
- Only analyze information explicitly mentioned in the provided article
- If information is not available for a section, use "null" or empty arrays
- Maintain consistency in entity naming (e.g., "Walmart" not "Wal-Mart")
- Prioritize actionable insights over general observations




### Input Articles:
```{articles_text}``` 
"""
###################

# Measuring latency
start_time = time.time()

################
chat_completion = client.chat.completions.create(
    model="deepseek-r1-distill-llama-70b",
    messages=[{"role":"user","content":prompt_template}],
    #prompt=prompt_template,
    max_tokens=8096,
    top_p=0.5,
    temperature=0.3
)

#######################
# Calculate latency and total processing time
latency = time.time() - start_time
#print(chat_completion.choices[0].message.content)
output_text = chat_completion.choices[0].message.content


# Number of tokens generated in the response
output_tokens = len(output_text.split())


# Calculate tokens for input and output
input_tokens = tokenizer(prompt_template)['input_ids']
output_tokens = tokenizer(output_text)['input_ids']

# Calculate the number of tokens
num_input_tokens = len(input_tokens)
num_output_tokens = len(output_tokens)

# Calculate word length for input and output
input_word_count = len(prompt_template.split())
output_word_count = len(output_text.split())


# Price calculation

# Price in USD per million input and output  tokens
price_per_million_input_tokens = 0.75  
price_per_million_output_tokens = 0.99  


# Calculation of entry and exit costs
input_cost = (num_input_tokens / 1_000_000) * price_per_million_input_tokens
output_cost = (num_output_tokens / 1_000_000) * price_per_million_output_tokens
total_cost = input_cost + output_cost


# Define the folder path and create folders if necessary
output_dir = os.path.join("Output", "Ergebnis_test")
os.makedirs(output_dir, exist_ok=True)

# Define the file name with the current date and time
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
file_name = f"{current_time}_zs_deepseek4.txt"
file_path = os.path.join(output_dir, file_name)

# Save the result in the text file
with open(file_path, "w") as file:
    file.write(output_text)
    

#print(f"Output saved to {file_path}")
print(f"Output saved to {file_path}")


# Function to save the analysis in an Excel file

def save_analysis_to_excel(model_name, context_window, output_speed, latency, price, output_price, input_price, time_period, num_input_tokens, num_output_tokens, input_word_count, output_word_count, base_dir="LLM_Analysis"):
    data = {
        "Context Window": [context_window],
        "Output Speed (tokens/s)": [output_speed],
        "Latency (seconds)": [latency],
        "Total Cost ($)": [price],
        "Output Price ($ per M tokens)": [output_price],
        "Input Price ($ per M tokens)": [input_price],
        "Time Period (seconds)": [time_period],
        "Input Tokens": [num_input_tokens],
        "Output Tokens": [num_output_tokens],
        "Input Word Count": [input_word_count],
        "Output Word Count": [output_word_count]
    }
    
    df = pd.DataFrame(data)
    # Create the path to the Excel file
    output_dir = os.path.join(base_dir, model_name)
    os.makedirs(output_dir, exist_ok=True)
    file_name = f"{model_name}_analysis_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.xlsx"
    file_path = os.path.join(output_dir, file_name)
    try:
        df.to_excel(file_path, sheet_name=model_name[:31], index=False)
    except Exception as e:
        print(f"An error occurred: {e}")
           
        
            

# An example of use
context_window = 131072  # Example of the size of the context window
output_speed = num_output_tokens / latency  # Tokens per second
time_period = latency  # Total time in seconds

model_name = f"zs_deepseek{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
save_analysis_to_excel(
    model_name=model_name,
    context_window=context_window,
    output_speed=output_speed,
    latency=latency,
    price=total_cost,
    output_price=price_per_million_output_tokens,
    input_price=price_per_million_input_tokens,
    time_period=time_period,
    num_input_tokens=num_input_tokens,
    num_output_tokens=num_output_tokens,
    input_word_count=input_word_count,
    output_word_count=output_word_count
)





2025-07-06 16:05:48,325 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Output saved to Output\Ergebnis_test\2025-07-06_16-05-48_zs_deepseek4.txt


## qwen/qwen3-32b

In [ ]:
def get_articles_with_metadata(df, num_articles=5, offset=0):
    articles = df.iloc[offset:offset + num_articles][['url',
 'title',
 'publish_date',
 'authors',
 'scraped_at',
 'keywords',
 'text',
 'text_length']].to_dict(orient='records')
    return articles

# Example to obtain the first 5 articles
articles = get_articles_with_metadata(df, num_articles=5, offset=0)

# Using metadata to build article text
articles_text = "\n\n".join(
    [f"### Article {i+1}:\nTitle: {article['title']}\nURL: {article['url']}\nPublish_date: {article['publish_date']}\n"
     for i, article in enumerate(articles)]
)

# Dynamically generate the ‘Article Summaries’ section with numbers
article_summaries = "\n\n".join(
    [f"""### {i+1}. 
     
     **Title**: {article['title']}
   **Link to source**: {article['url']} 
   **Publish_date**: {article['publish_date']}
   **Summary**: Summarize the key points of the article in 2-3 sentences.
   **Category**: Categorize the article into one or more retail-focused categories:["product_innovation", "pricing_strategy", "market_expansion", "retail_tech", "sustainability", "supply_chain_risk", "competitor_moves", "consumer_trends", "others"].
   **Entity Extraction**: Extract and identify brands/compagnies,  products, regions/markets, Financial Data, dates
   **sentiments**:Analyze the overall sentiment: Positive, Negative, or Neutral, Market Sentiment, Competitive Sentiment, Supply Chain Sentiment. Provide a brief justification.
   **trend detection **:Identify whether the article includes any of the following:Product launches or innovations, Pricing or promotion strategies, Store openings, expansions, or mergers, Retail technology integrations, Sustainability initiatives, Supply chain risks: delays, shortages, geopolitical events
   **predictive intelligence **:Predict whether the article suggests:Short-term Implications, Medium-term Trends, Long-term Forecasts, Recommendation Priority, An emerging trend, A supply chain risk, A market shift, Or none
   **Business Impact Scoring**: Rate the potential impact of the article on the business on a scale of 1 to 10, where 1 is low impact and 10 is high impact. Provide a brief justification.
   
   
   """
     for i, article in enumerate(articles)]
)

# Create prompt
prompt_template = f"""
You are an expert retail intelligence analyst working for a major retail/FMCG company that serves 15,000 suppliers and 200 retailers. Your role is to analyze news articles and extract actionable business intelligence that will help stakeholders make informed decisions about competitive positioning, supply chain management, and market opportunities.

## ROLE
Act as a Senior Retail Intelligence Analyst with expertise in:
- Competitive intelligence and market analysis
- Supply chain risk assessment
- Predictive trend analysis
- FMCG/Retail industry dynamics
- Business impact evaluation

## ACTION
Analyze the provided news article and perform the following comprehensive analysis:

### 1. INTELLIGENT CLASSIFICATION
Categorize the article into ONE primary category:
- **product_innovation**: New products, launches, and innovations
- **pricing_strategy**: Pricing strategies, promotions, and competitive pricing
- **market_expansion**: Store openings, market expansion, and geographic growth
- **supply_chain**: Supply chain disruptions, logistics, and sourcing issues
- **technology_retail**: Retail technology, digital transformation, and e-commerce
- **sustainability**: Sustainability initiatives and ESG practices
- **consumer_behavior**: Consumer trends, preferences, and behavioral shifts
- **regulatory_compliance**: Regulatory changes, compliance, and policy impacts
- **competitive_intelligence**: Competitor moves, market share, and strategic partnerships
- **risk_management**: Geopolitical risks, economic factors, and crisis management

### 2. ENTITY EXTRACTION
Extract and identify:
- **Brands/Companies**: All mentioned retail brands, suppliers, competitors
- **Products**: Specific products, categories, or product lines
- **Regions/Markets**: Geographic locations, countries, cities, markets
- **Key Figures**: Important people, executives, industry leaders
- **Financial Data**: Revenue, sales figures, market share, investments
- **Dates**: Important dates, timelines, deadlines

### 3. RETAIL-SPECIFIC SENTIMENT ANALYSIS
Assess sentiment across three dimensions:
- **Market Sentiment**: Positive/Negative/Neutral impact on market conditions
- **Competitive Sentiment**: Advantage/Disadvantage/Neutral for competitive positioning
- **Supply Chain Sentiment**: Stable/Risk/Opportunity for supply chain operations

### 4. TREND DETECTION & INNOVATION ANALYSIS
Identify:
- **Emerging Trends**: New consumer behaviors, market shifts, technological adoption
- **Innovation Signals**: R&D developments, patent filings, breakthrough technologies
- **Product Launches**: New product introductions, line extensions, category expansions
- **Market Disruptions**: Business model changes, new market entrants, industry shifts

### 5. SUPPLY CHAIN RISK ASSESSMENT
Evaluate:
- **Disruption Risks**: Logistics issues, transportation problems, warehouse challenges
- **Supplier Issues**: Supplier bankruptcies, quality problems, capacity constraints
- **Geopolitical Risks**: Trade tensions, sanctions, regulatory changes
- **Economic Factors**: Inflation, currency fluctuations, commodity price changes

### 6. PREDICTIVE INTELLIGENCE
Based on the information, predict:
- **Short-term Implications** (1-3 months): Immediate business impacts
- **Medium-term Trends** (3-12 months): Evolving market dynamics
- **Long-term Forecasts** (1-3 years): Strategic industry shifts
- **Recommendation Priority**: High/Medium/Low urgency for stakeholder action

### 7. BUSINESS IMPACT SCORING
Rate the criticality on a scale of 1-10:
- **Competitive Impact**: How significantly this affects competitive positioning
- **Supply Chain Impact**: Risk level for supply chain operations
- **Market Opportunity**: Potential for new business opportunities
- **Strategic Importance**: Overall relevance to business strategy


## FORMAT

### Here's how I'd like you to return my results:

Dear Customer,

Welcome to our weekly newsletter, where we provide you with a brief overview summary of significant events. 

### Article Summaries:


{article_summaries}

Stay tuned for our next issue as we delve deeper into the evolving challenges and opportunities shaping the retail landscape.

Warm regards,
Your retail.ai team



##There are several aspects we look for in a good summary:
 **Consistency**: A good summary should be consistent with the article. Please do not add any new information to the summary, even if you know more about the article's content. For example, if the article does mention the first name of 'President Biden', please do not use the full name 'Joe Biden' in your summary.
 **Relevance**: A good summary should only include important information from the article. In general, you should include the key points you highlighted in your summary, but feel free to omit details you think are unnecessary to convey the main idea of the article.
 **Conciseness**: A good summary should be short and to the point. But it is okay if the summary is shorter when the article is short.
 **Coherence**: A good summary is more than just a list of sentences. It should be a coherent paragraph that makes sense.


## TONE
Maintain a professional, analytical, and strategic tone throughout your analysis. Be:
- **Objective**: Base conclusions on factual evidence from the text
- **Actionable**: Provide insights that can drive business decisions
- **Comprehensive**: Cover all relevant aspects without being verbose
- **Strategic**: Focus on business implications and competitive advantages
- **Precise**: Use specific retail/FMCG terminology and metrics


## CONSTRAINTS
- Only analyze information explicitly mentioned in the provided article
- If information is not available for a section, use "null" or empty arrays
- Maintain consistency in entity naming (e.g., "Walmart" not "Wal-Mart")
- Prioritize actionable insights over general observations




### Input Articles:
```{articles_text}``` 
"""
###################

# Measuring latency
start_time = time.time()

################
chat_completion = client.chat.completions.create(
    model="deepseek-r1-distill-llama-70b",
    messages=[{"role":"user","content":prompt_template}],
    #prompt=prompt_template,
    max_tokens=8096,
    top_p=0.5,
    temperature=0.3
)

#######################
# Calculate latency and total processing time
latency = time.time() - start_time
#print(chat_completion.choices[0].message.content)
output_text = chat_completion.choices[0].message.content


# Number of tokens generated in the response
output_tokens = len(output_text.split())


# Calculate tokens for input and output
input_tokens = tokenizer(prompt_template)['input_ids']
output_tokens = tokenizer(output_text)['input_ids']

# Calculate the number of tokens
num_input_tokens = len(input_tokens)
num_output_tokens = len(output_tokens)

# Calculate word length for input and output
input_word_count = len(prompt_template.split())
output_word_count = len(output_text.split())


# Price calculation

# Price in USD per million input and output  tokens
price_per_million_input_tokens = 0.29  
price_per_million_output_tokens = 0.59  


# Calculation of entry and exit costs
input_cost = (num_input_tokens / 1_000_000) * price_per_million_input_tokens
output_cost = (num_output_tokens / 1_000_000) * price_per_million_output_tokens
total_cost = input_cost + output_cost


# Define the folder path and create folders if necessary
output_dir = os.path.join("Output", "Ergebnis_test")
os.makedirs(output_dir, exist_ok=True)

# Define the file name with the current date and time
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
file_name = f"{current_time}_zs_qwen.txt"
file_path = os.path.join(output_dir, file_name)

# Save the result in the text file
with open(file_path, "w") as file:
    file.write(output_text)
    

#print(f"Output saved to {file_path}")
print(f"Output saved to {file_path}")


# Function to save the analysis in an Excel file

def save_analysis_to_excel(model_name, context_window, output_speed, latency, price, output_price, input_price, time_period, num_input_tokens, num_output_tokens, input_word_count, output_word_count, base_dir="LLM_Analysis"):
    data = {
        "Context Window": [context_window],
        "Output Speed (tokens/s)": [output_speed],
        "Latency (seconds)": [latency],
        "Total Cost ($)": [price],
        "Output Price ($ per M tokens)": [output_price],
        "Input Price ($ per M tokens)": [input_price],
        "Time Period (seconds)": [time_period],
        "Input Tokens": [num_input_tokens],
        "Output Tokens": [num_output_tokens],
        "Input Word Count": [input_word_count],
        "Output Word Count": [output_word_count]
    }
    
    df = pd.DataFrame(data)
    # Create the path to the Excel file
    output_dir = os.path.join(base_dir, model_name)
    os.makedirs(output_dir, exist_ok=True)
    file_name = f"{model_name}_analysis_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.xlsx"
    file_path = os.path.join(output_dir, file_name)
    try:
        df.to_excel(file_path, sheet_name=model_name[:31], index=False)
    except Exception as e:
        print(f"An error occurred: {e}")
           
        
            

# An example of use
context_window = 131072  # Example of the size of the context window
output_speed = num_output_tokens / latency  # Tokens per second
time_period = latency  # Total time in seconds

model_name = f"zs_qwen{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
save_analysis_to_excel(
    model_name=model_name,
    context_window=context_window,
    output_speed=output_speed,
    latency=latency,
    price=total_cost,
    output_price=price_per_million_output_tokens,
    input_price=price_per_million_input_tokens,
    time_period=time_period,
    num_input_tokens=num_input_tokens,
    num_output_tokens=num_output_tokens,
    input_word_count=input_word_count,
    output_word_count=output_word_count
)





2025-07-06 16:09:53,908 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Output saved to Output\Ergebnis_test\2025-07-06_16-09-54_zs_qwen.txt
